# Programming Assignment 2: Movie recommendation

- 과제 목표: 뉴럴 네트워크 모델을 설계한 후 모델을 학습하여 각 영화들의 embedding 들을 생성하고, 영화 embedding 을 활용하여 각 사용자에게 맞춤형 영화를 추천

# Notice

<br>

- 과제를 수행하면서 각 task 마다 꼭 주어진 1개의 cell만을 사용할 필요는 없으며, 여러 개의 cell을 추가하여 자유롭게 사용해도 괜찮습니다.
- 과제 수행을 위해 필요한 module이 있다면 추가로 import 해도 괜찮습니다.

# Import Modules

In [1]:
import warnings, random
import numpy as np
import pandas as pd
import random
import torch # For building network
import torch.nn as nn
import torch.nn.functional as F
from itertools import permutations
from itertools import permutations # For making pairs
warnings.filterwarnings('ignore')

In [2]:
import sys
print(sys.executable)

/Users/vidigummy/opt/anaconda3/envs/RS/bin/python


# Data loading

In [3]:
dir = './MovieLens100K/'
df_ratings = pd.read_csv(dir + 'ratings.csv', usecols=['userId', 'movieId', 'rating'])
df_movies_real = pd.read_csv(dir + 'movies.csv', usecols=['movieId', 'title', 'genres']) # for title-matching

# Preprocessing data

<br>

> ### Problem 1 (3 points)

<br>

1. df_ratings로 부터 각 사용자들이 본 영화를 기록.
2. 사용자 마다 본 영화 목록을 $(movie1, movie2)$, $(movie2, movie1)$ 과 같이 pair로 생성.
    - 즉, 각 사용자 마다 본 영화 목록에 대해 Permutation을 수행
3. 2번 과정이 끝난 후, random을 이용해 각 pair 순서를 무작위로 shuffle.

In [4]:
#### Your Code Here
df_users = df_ratings['userId'].drop_duplicates()
movies_permutation_result = []
df_movies = df_ratings['movieId'] < 1000
df_movies_true = df_ratings[df_movies]
print(df_movies_true)
for i in df_users:
    is_user_movies = df_movies_true['userId'] == i
    user_movies = df_movies_true[is_user_movies]['movieId']
    user_movies_list = user_movies.values.tolist()
    user_movies_permutation = list(permutations(user_movies_list, 2))
    movies_permutation_result.extend(user_movies_permutation)
random.shuffle(movies_permutation_result)
# print(len(movies_permutation_result))
# print(movies_permutation_result)
# print(len(movies_permutation_result))

       userId  movieId  rating
0           1        1     4.0
1           1        3     4.0
2           1        6     4.0
3           1       47     5.0
4           1       50     5.0
...       ...      ...     ...
99590     610      912     3.5
99591     610      919     3.5
99592     610      923     4.0
99593     610      924     4.5
99594     610      968     4.0

[21667 rows x 3 columns]


# Build and train neural networks for generating movie embeddings

<br>

> ### Problem 2 (4 points)

<br>

- 각 영화 임베딩을 구하기 위해 뉴럴 네트워크 모델을 활용하여 multi-class classification 을 수행

- 설계할 신경망의 기본 구조는 **Input Layer - Hidden(Embedding) Layer - Output Layer**.
    - 7주차 강의자료 p.7 신경망 구조 이미지 참고

- 현재 Network를 통해 하고자 하는 task는 multi-class classification.
    - 예: $(movie1, movie2)$ 와 같은 입력 데이터와 정답 출력 데이터를 이용해 모델을 학습
        - Input : $movie1$의 one-hot vector
        - Output : $\widehat{movie2}$의 one-hot vector
        - Compute Loss : $\widehat{movie2}$ 와 $movie2$ 간의 Cross-entropy Loss
- 학습이 완료된 이후에 input layer와 hidden(embedding) layer 사이의 weight matrix $W_{in}$를 movie에 대한 embedding vector로 사용이 가능.
> embedding size(# of hidden units)는 100 이하로 두는 것을 권장. <br>
> embedding layer 다음 hidden layer를 더 추가하여 Genre와 같은 추가 정보를 학습에 활용 할 수도 있음 (필수적으로 고려해야할 사항은 아님).

- 설계한 뉴럴 네트워크 모델의 학습이 완료된 후, 학습된 weight matrix $W_{in}$의 행/열벡터를 각 영화에 대한 embedding vector로 간주하여 영화 embedding 들을 구할 수 있음.

In [5]:
class NnNetwork(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.embedded_layer = nn.Linear(input_size, 100)
        self.hidden_layer = nn.Linear(100, 100)
        self.output_layer = nn.Linear(100,input_size)
        self.relu = nn.ReLU()
        self.activation = nn.Sigmoid()
    def forward(self,x):
        x = self.embedded_layer(x)
        x = self.relu(x)
        x = self.hidden_layer(x)
        x = self.relu(x)
        x = self.output_layer(x)
        x = self.activation(x)
        return x

In [6]:
#### Your Code Here
movies_pair_dataframe = pd.DataFrame(data= movies_permutation_result, columns=['movie1','movie2'])

#영화의 개수를 가지고 one-hot vector로 만들어뿌자
df_movie1 = pd.DataFrame(data=movies_pair_dataframe['movie1'].values,columns=['movieId'])
print(df_movie1)
df_movie2 = pd.DataFrame(data=movies_pair_dataframe['movie2'].values,columns=['movieId'])
df_concat_movies = pd.concat([df_movie1,df_movie2],axis=0)
df_one_hot_embedded = pd.get_dummies(data=df_concat_movies, columns = ['movieId'])
df_one_hot_embedded_movie1 = df_one_hot_embedded.iloc[:int(df_one_hot_embedded.shape[0]/2), :]
df_one_hot_embedded_movie2 = df_one_hot_embedded.iloc[int(df_one_hot_embedded.shape[0]/2):,:]
#df로 만들었다. (1099200, 1859)로 자식들이 아주 건강하다.
torch_one_hot_movie1 = torch.as_tensor(df_one_hot_embedded_movie1.values).float()
torch_one_hot_movie2 = torch.as_tensor(df_one_hot_embedded_movie2.values).float()
print("torch_one_hot_movie1 shape : ", torch_one_hot_movie1.shape)
model = NnNetwork(torch_one_hot_movie1.shape[1])
loss_function = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
# optimizer2 = torch.optim.Adam(model.parameters(),lr=0.1, betas = (0.9,0.999), eps=1e-08, weight_decay=0, amsgrad=False)
optimizer2 = torch.optim.Adam(model.parameters(),lr=0.1, betas = (0.9,0.999), eps=1e-08, weight_decay=0, amsgrad=False)

for epoch in range(2):
    #make one-hot vector(input, target Generation)
    output = model(torch_one_hot_movie1)
    loss = loss_function(output, torch_one_hot_movie2)
    optimizer2.zero_grad()
    loss.backward()
    optimizer2.step()
    # loss.backward()
    print('Epoch {:4d}/{} loss: {:.6f}'.format(
        epoch+1, 2, loss.item()
    ))
    



         movieId
0            155
1             72
2            553
3            508
4            775
...          ...
1869541      920
1869542      741
1869543      587
1869544      329
1869545      973

[1869546 rows x 1 columns]
torch_one_hot_movie1 shape :  torch.Size([1869546, 761])
Epoch    1/2 loss: 6.635469


In [ ]:
movie_embedding = dict()
cnt = 0
# print(df_movie1)
#761*100
for param in model.named_parameters():
    a = param[1]
    print(len(a[0]))
    print(len(a))
    for i in range(len(a[0])):
        # print(i)
        # if(int(df_movie1.loc[i,'movieId']) == 70):
        #     print(70)
        print(int(df_movie1.loc[i,'movieId']))
        movie_embedding[int(df_movie1.loc[i,'movieId'])] = a[:,i]
    break
print(len(movie_embedding))
# print(movie_embedding)


761
100
364
57
474
454
485
521
1
141
810
318
257
719
158
590
426
805
608
558
1
302
704
903
569
912
223
47
802
780
736
800
780
942
342
930
158
367
529
18
172
838
592
924
24
661
303
198
915
23
788
367
231
260
32
344
145
207
296
317
719
318
736
183
926
281
288
11
216
307
231
203
34
673
315
356
87
34
589
667
588
75
648
110
223
457
457
224
538
535
490
160
225
595
261
329
296
491
260
656
356
302
475
475
635
356
47
529
372
538
89
552
410
896
968
367
317
112
339
300
231
412
707
41
969
58
150
414
711
32
73
474
317
945
374
316
173
95
952
377
242
953
168
432
16
349
838
362
616
528
223
608
110
150
34
858
252
296
507
553
553
22
617
222
173
266
160
493
95
628
529
111
380
587
253
932
534
208
879
57
168
173
1
16
527
367
926
105
491
355
44
186
231
357
92
168
541
433
647
47
903
317
110
66
968
70
551
45
325
380
279
393
520
605
832
635
585
965
318
909
736
65
198
440
158
380
292
79
409
364
761
788
922
247
903
802
82
5
293
353
288
261
145
479
574
17
508
781
39
110
897
296
520
947
564
954
517
160
10
318
593


# Recommend customized movies to user

<br>

> ### Problem 3 (3 points)

<br>

- 임의의 한명의 사용자에 대하여 해당 사용자가 봤던 영화 n개에 대해 **통합된 embedding vector**를 생성.
    - n개의 embedding vector들에 대해, element-wise한 계산을 통해 통합된 하나의 embedding vector를 생성.
    - 이 embedding vector는 해당 사용자의 전반적인 영화 시청 성향을 나타내는 embedding vector로 간주할 수 있음.
    - 즉, **사용자 1명 당 1개의 embedding vector**를 가짐.
- 통합된 embedding vector와 학습된 weight matrix $W_{in}$의 모든 영화 embedding vector들 간의 유사도를 계산.
- 그 중 유사도가 높은 (top n) 영화들을 선정, 사용자에게 추천.
    > Recommended format : MovieId, Title, Genre, Similarity 가 포함된 형식

In [ ]:
#### Your Code Here
is_user_movies = df_movies_true['userId'] == 1
users_movies = df_movies_true[is_user_movies]
user_movies = users_movies['movieId']
# print(user_movies.shape)
user_movie_dict =dict()
# print(df_movie1[df_movie1['movieId'] == 157])
# for movie in movie_embedding:
#     print(movie)
for movieId in user_movies:
    # print(movie_embedding[movieId])
    try:
        user_movie_dict[movieId] = movie_embedding[movieId]
    except:
        print(movieId , " has lost!")
# print(user_movie_dict)
print(len(user_movie_dict))
user_embedding_vector = torch.zeros(100)
print(user_embedding_vector)
for i in user_movie_dict.keys():
    # print(len(i_list))
    i_list = user_movie_dict[i]/len(user_movie_dict)
    for j in range(0,len(i_list)):
        user_embedding_vector[j] += i_list[j]

#유저 임베딩 벡터 완성(이게 맞나...)
print(user_embedding_vector)



157  has lost!
163  has lost!
423  has lost!
441  has lost!
943  has lost!
44
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])
tensor([ 0.0967, -0.0617,  0.0765, -0.0654, -0.0136, -0.0791, -0.0399,  0.0945,
         0.0942, -0.0122,  0.0873, -0.0109, -0.0040, -0.0312, -0.1105,  0.1167,
         0.1140, -0.0076,  0.0039,  0.0717,  0.1102, -0.0535,  0.0680,  0.0670,
         0.0834, -0.0064, -0.0597,  0.0823,  0.1260, -0.0479, -0.1035, -0.0413,
        -0.0071,  0.0853,  0.0965, -0.0903,  0.1218,  0.0788, -0.0795,  0.0931,
         0.0767,  0.1232,  0.0723,  0.1404, -0.0098, -0.0414,  0.0743,  0.0905,
 

In [ ]:
# print(movie_embedding)
result =dict()
for embedded_movie in movie_embedding.keys():
    cosine_sim = F.cosine_similarity(movie_embedding[embedded_movie], user_embedding_vector, dim = 0)
    # print(embedded_movie, " : ",cosine_sim)
    result[embedded_movie] = cosine_sim

result_recommend_list = sorted(result.items(), reverse=True, key=lambda item: item[1])
# print(result_recommend_dict)
# for i in result_recommend_dict:
#     print(i[0])
result_list_top_ten = list()
result_list_top_ten_sim = list()
result_recommend_list_top_ten = result_recommend_list[0:10]
for i in result_recommend_list_top_ten:
    result_list_top_ten.append(i[0])
    result_list_top_ten_sim.append(i[1])
# print(result_list_top_ten)
# print(df_movies_real[df_movies_real['movieId'].isin(result_list_top_ten)])
for ten in range (0,len(result_list_top_ten)):
    one_thing_title = df_movies_real[df_movies_real['movieId'] == result_list_top_ten[ten]]['title'].values[0]
    one_thing_genres = df_movies_real[df_movies_real['movieId'] == result_list_top_ten[ten]]['genres'].values
    # print(ten," ", one_thing_title, " ", one_thing_genres)
    print("movieId : ",result_list_top_ten[ten], " | title : ", one_thing_title, "| genres : ",one_thing_genres,"| sim :", result_list_top_ten_sim[ten].item())



movieId :  942  | title :  Laura (1944) | genres :  ['Crime|Film-Noir|Mystery'] | sim : tensor(0.8773, grad_fn=<DivBackward0>)
movieId :  736  | title :  Twister (1996) | genres :  ['Action|Adventure|Romance|Thriller'] | sim : tensor(0.8704, grad_fn=<DivBackward0>)
movieId :  354  | title :  Cobb (1994) | genres :  ['Drama'] | sim : tensor(0.8634, grad_fn=<DivBackward0>)
movieId :  348  | title :  Bullets Over Broadway (1994) | genres :  ['Comedy'] | sim : tensor(0.8628, grad_fn=<DivBackward0>)
movieId :  75  | title :  Big Bully (1996) | genres :  ['Comedy|Drama'] | sim : tensor(0.8573, grad_fn=<DivBackward0>)
movieId :  346  | title :  Backbeat (1993) | genres :  ['Drama|Musical'] | sim : tensor(0.8565, grad_fn=<DivBackward0>)
movieId :  537  | title :  Sirens (1994) | genres :  ['Drama'] | sim : tensor(0.8550, grad_fn=<DivBackward0>)
movieId :  319  | title :  Shallow Grave (1994) | genres :  ['Comedy|Drama|Thriller'] | sim : tensor(0.8549, grad_fn=<DivBackward0>)
movieId :  123  | 

잘 모르겠습니다. 죄송합니다. 부끄러운 말씀이지만, 무엇을 해야할 지에 대해서는 이해했습니다. 하지만 머신러닝을 위한 python 코드는 처음 짜는 것이기도 하고, 어떤 것이 좋은지도 전혀 알지 못하는 상황에서 이 과제를 해결하기에는 능력이 부족하다는 판단입니다. 
분명 오랜 시간을 투자했고, 여러모로 찾아봤지만 제 능력과 시간 내에서는 해결을 할 수 없다는 결론입니다.
다음 과제가 나올지는 잘 모르겠으나, 다음에는 중간 과정마다 어떠한 결과가 나오는 것이 좋은 것일지 알려주실 수 있을까요? 이또한 무례한 것이라면 사과 드리겠으나, 머신러닝 코드 에 대한 이해가 부족한 저로서는 이것이 최소한의 요청일듯합니다. 죄송합니다.